### Import relevant libraries

In [2]:
import os
import pandas as pd
import numpy as np

from nilearn import image
from nilearn import input_data
from nilearn import signal
from nilearn import connectome

from bids import BIDSLayout
from load_confounds import Params9
from load_confounds import AnatCompCor

import process_bids as pb
import process_connectome as pc

pd.options.mode.chained_assignment = None

### Useful Functions

In [1]:
# Inputs: dataset.
dataset_name = 'msc'
dataset_path = os.path.abspath('../msc-raw')
dataset_derivatives_path = os.path.abspath(r'../msc-derivatives/fmriprep')

# Inputs: brain parcellation.
atlas_name = 'CAB-NP'
atlas_path = os.path.abspath(r'../atlas/CAB-NP_volumetric/CAB-NP_volumetric_liberal.nii.gz')

# Settings.
pybids_cache_path = 'pybids_True.sql'
reset_pybids_cache = False
validate_bids = True
confounds_algorithms = {
    'None': None,
    'params9': Params9(),
    'aCompCor': AnatCompCor()
}

# Outputs.
denoised_output_dir_tpl = f'{dataset_name}-derivatives/denoised-v0.01/sub-{{sub_id}}/'
denoised_output_csv_tpl = f'{dataset_name}-derivatives/denoised-v0.01/sub-{{sub_id}}/{{atlas_name}}-denoised-timeseries.csv'
denoised_output_info_csv = f'{dataset_name}-derivatives/denoised-v0.01/denoised-timeseries-info.csv'
connectomes_output_dir_tpl = f'{dataset_name}-derivatives/connectomes-v0.01/sub-{{sub_id}}/'
connectomes_output_csv_tpl = f'{dataset_name}-derivatives/connectomes-v0.01/sub-{{sub_id}}/{{atlas_name}}-{{confound-name}}-{{caract}}.csv'

### Load input dataset bids structure

In [5]:
# Lecture du format BIDS
layout = BIDSLayout(dataset_path, derivatives = dataset_derivatives_path, validate=validate_bids,
                    database_path=pybids_cache_path, index_metadata=True, reset_database=reset_pybids_cache)

layout

BIDS Layout: ...D:\msc-raw | Subjects: 0 | Sessions: 0 | Runs: 0

### Load input dataset subject list

In [6]:
sub_ids = layout.get_subjects()
print('Subjects ID:', sub_ids)

Subjects ID: ['MSC01', 'MSC02', 'MSC03', 'MSC04', 'MSC05', 'MSC06', 'MSC07', 'MSC08', 'MSC09', 'MSC10']


### Parallel processing for time series denoising calculation

In [7]:
time_series_info = pd.DataFrame(columns = sub_ids, index = ['path','nlevels','atlas', 'tr'])
time_series_info.loc['atlas', :] = atlas_name

def get_tr_by_subject(sub_id):
    try:
        return layout.get_tr(subject=sub_id)
    except Exception:
        return None
time_series_info.loc['tr', :] = time_series_info.columns.map(get_tr_by_subject)

In [ ]:
for sub_id in sub_ids:
    denoised_output_csv = denoised_output_csv_tpl.format(sub_id=sub_id, atlas_name=atlas_name)
    
    ses_ids = pb.get_sessions(layout, sub_id)
    task_ids_by_ses_ids = pb.get_tasks(layout, sub_id, ses_ids)
    run_ids_by_tasks_ids_by_ses_ids = pb.get_run(layout, sub_id, ses_ids, task_ids_by_ses_ids)
    sub_timeseries_df = pc.create_empty_df_timesseries(ses_ids, task_ids_by_ses_ids, run_ids_by_tasks_ids_by_ses_ids, confounds_algorithms.keys())

    # Calcul sur l'ensemble des nifti
    sub_bold_nifti_paths = layout.get(subject=sub_id, extension='nii.gz', suffix='bold', scope='derivatives',
                                  return_type='filename')

    for confounds_algorithm_name, confounds_algorithm in confounds_algorithms.items():
        loaded_confounds = confounds_algorithm.load(sub_bold_nifti_paths)

        time_series_info.loc['path', sub_id] = denoised_output_csv
        time_series_info.loc['nlevels', sub_id] = sub_timeseries_df.columns.nlevels
        t_r = time_series_info.loc['tr', sub_id]
        if t_r == None:
            print (f'error: subject {sub_id} cancelled')
            print (f'error: multiple tr within subject')
            continue
        atlas_masker = input_data.NiftiLabelsMasker(atlas_path, standardize=False, smoothing_fwhm=6, low_pass=0.01, t_r=t_r)
        sub_timeseries_df = pc.calculate_timeseries(atlas_masker, run_ids_by_tasks_ids_by_ses_ids, sub_bold_nifti_paths, denoised_output_csv,
                                                    sub_timeseries_df, t_r, 
                                                    confound_name=confounds_algorithm_name, loaded_confounds=loaded_confounds)
                                                    
    time_series_info.to_csv(denoised_output_info_csv, header=True)
    time_series_info

### Subjects caracteristics

In [ ]:
time_series_info = pd.read_csv(info_csv_path, header = 0, index_col=0)
time_series_info

### Connectomes calculation

In [ ]:
confounds_algorithms = {
    'params9': Params9(),
    'aCompCor': AnatCompCor()
}

# Set pairs of atlas / confound algorithms we want to use.
output_pairs = ((atlas_name, confounds_algorithm_name) for confounds_algorithm_name in confounds_algorithm.keys())
for atlas_name, confounds_algorithm_name in output_pairs:
    # Create output directory for every pair.
    connectomes_output_dir = connectomes_output_dir_tpl.format(atlas_name=atlas_name)
    if not os.path.exists(connectomes_output_dir):
        os.makedirs(connectomes_output_dir)
    
    # Extract connectomes in the output directory using the given settings.
    pc.extract_connectomes(
        time_series_info,
        connectomes_output_dir,
        confounds_algorithm_name,
        kind='correlation',
        vectorize=True,
        discard_diagonal=True
    )